In [5]:
from flask import Flask, request, jsonify
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import os
from gemini_api import GeminiAPI  # Placeholder for the Gemini 1.5 Pro API wrapper

# Initialize Flask App
app = Flask(__name__)

# Load Models
text_tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Example model
text_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Initialize Gemini API (Assumed wrapper provided)
gemini_api = GeminiAPI(api_key="YOUR_GEMINI_API_KEY")

# Define Preprocessing Functions
def preprocess_text(input_text):
    return text_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Define Prompts
def get_prompts():
    prompts = [
        "Analyze the provided city description and suggest potential vacant spaces that could be converted into green areas. Consider factors like the current land usage, population density, and proximity to urban areas.",
        "From the identified vacant spaces, suggest the best location for conversion into a green space. Provide justifications based on accessibility, sustainability, and environmental impact.",
        "Examine the city's layout in the description and suggest the best transportation methods to reach the proposed green space. Consider accessibility by public transport, cycling, or walking.",
        "Propose sustainable infrastructure for the green space, considering local geography, climate conditions, and potential for community use.",
        "Identify the best location for a community-centered green space, considering factors like ease of access and potential for social interaction.",
        "Analyze the provided building blueprint and suggest modifications to improve energy efficiency. Consider factors like insulation, window placement, and the potential for integrating renewable energy sources like solar panels.",
        "Examine the traffic flow throughout the city and recommend strategies to reduce traffic congestion and air pollution. Consider promoting public transportation use, creating car-free zones, and optimizing traffic light timing.",
        "Identify areas with high rainwater runoff and propose solutions for sustainable water management. Consider implementing green roofs, permeable pavements, and rainwater harvesting systems.",
        "Analyze the city's waste management system and suggest improvements to reduce waste generation and increase recycling rates. Consider promoting composting programs, implementing extended producer responsibility schemes, and educating residents about waste reduction strategies.",
        "Evaluate the city's noise pollution levels and recommend strategies to create quieter and more peaceful urban environments. Consider installing noise barriers, promoting the use of electric vehicles, and designating quiet zones in residential areas."
    ]
    return prompts

# Validation Functions
def validate_output(design_suggestions):
    # Placeholder for validation logic
    validated_results = []
    for suggestion in design_suggestions:
        # Example validation logic: Check if essential elements like sustainability metrics are present
        if "sustainability" in suggestion.lower() or "biodiversity" in suggestion.lower():
            validated_results.append({"suggestion": suggestion, "valid": True})
        else:
            validated_results.append({"suggestion": suggestion, "valid": False, "reason": "Lacks sustainability focus"})
    return validated_results

# Define Route for Design Suggestions
@app.route('/generate_design', methods=['POST'])
def generate_design():
    try:
        # Get JSON Input
        data = request.json

        # Extract Text Input
        description = data.get("description", "")

        # Preprocess Text Input
        text_inputs = preprocess_text(description)

        # Generate Text Features (No need for image-related features)
        text_features = text_model.generate(**text_inputs).tolist()

        # Combine Text Features (Image Features Removed)
        combined_features = {
            "text_features": text_features
        }

        # Query Gemini API with Only Text Features
        design_suggestions = gemini_api.get_design_suggestions(combined_features)

        # Validate Outputs
        validated_results = validate_output(design_suggestions)

        return jsonify({"design_suggestions": validated_results})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Define Route for Sample Prompts
@app.route('/sample_prompts', methods=['GET'])
def sample_prompts():
    return jsonify({"prompts": get_prompts()})

# Define Home Route
@app.route('/')
def home():
    return "Welcome to the Green Space AI App!"

# Run Flask App
if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'tensorflow'